In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Kidney_Clear_Cell_Carcinoma"
cohort = "GSE95425"

# Input paths
in_trait_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma"
in_cohort_dir = "../../input/GEO/Kidney_Clear_Cell_Carcinoma/GSE95425"

# Output paths
out_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/GSE95425.csv"
out_gene_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE95425.csv"
out_clinical_data_file = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/clinical_data/GSE95425.csv"
json_path = "../../output/preprocess/Kidney_Clear_Cell_Carcinoma/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Cell-type specific gene programs of the normal human nephron define kidney cancer subtypes"
!Series_summary	"Comprehensive transcriptome studies of cancers often rely on corresponding normal tissue samples to serve as a transcriptional reference. In this study we performed in-depth analyses of normal kidney tissue transcriptomes from TCGA and demonstrate that the histological variability in cellularity, inherent in the kidney architecture, lead to considerable transcriptional differences between samples. This should be considered when comparing expression profiles of normal and cancerous kidney tissues. We exploited these differences to define renal cell-specific gene signatures and used these as framework to analyze renal cell carcinoma (RCC) ontogeny. Chromophobe RCCs express FOXI1-driven genes that define collecting duct intercalated cells whereas HNF-regulated genes, specific for proximal tubule cells, are an integral part of clear cell and pa

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Optional, Callable, Dict, Any

# Examine the provided information to determine gene expression data availability
is_gene_available = True  # Dataset is likely to contain gene expression data based on the background info describing kidney tissue transcriptomes

# Analyze sample characteristics dictionary to identify keys containing trait, age, and gender information
trait_row = None  # No specific trait info available for kidney cancer in the samples (all are normal tissues)
age_row = None  # No age information available
gender_row = None  # No gender information available

# Define conversion functions (even though we don't have data for them, we define them for completeness)
def convert_trait(value: str) -> Optional[int]:
    """Convert trait values to binary format."""
    if not value or ":" not in value:
        return None
    
    val = value.split(":", 1)[1].strip().lower()
    
    # Since all samples are normal kidney tissue, there's no trait variation
    return None  # No appropriate conversion possible

def convert_age(value: str) -> Optional[float]:
    """Convert age values to continuous format."""
    if not value or ":" not in value:
        return None
    
    val = value.split(":", 1)[1].strip()
    try:
        return float(val)
    except ValueError:
        return None
    
def convert_gender(value: str) -> Optional[int]:
    """Convert gender values to binary format (0 for female, 1 for male)."""
    if not value or ":" not in value:
        return None
    
    val = value.split(":", 1)[1].strip().lower()
    if val in ["female", "f"]:
        return 0
    elif val in ["male", "m"]:
        return 1
    else:
        return None

# Determine trait availability
is_trait_available = False  # Since trait_row is None, trait data is not available

# Save metadata about the dataset using the validate_and_save_cohort_info function
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# Skip clinical feature extraction since trait_row is None


False

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the file paths for the SOFT file and matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Extract gene expression data from the matrix file
try:
    print("Extracting gene data from matrix file:")
    gene_data = get_genetic_data(matrix_file)
    if gene_data.empty:
        print("Extracted gene expression data is empty")
        is_gene_available = False
    else:
        print(f"Successfully extracted gene data with {len(gene_data.index)} rows")
        print("First 20 gene IDs:")
        print(gene_data.index[:20])
        is_gene_available = True
except Exception as e:
    print(f"Error extracting gene data: {e}")
    print("This dataset appears to have an empty or malformed gene expression matrix")
    is_gene_available = False

print(f"\nGene expression data available: {is_gene_available}")


Extracting gene data from matrix file:
Successfully extracted gene data with 27367 rows
First 20 gene IDs:
Index(['ILMN_1343291', 'ILMN_1343295', 'ILMN_1651209', 'ILMN_1651228',
       'ILMN_1651229', 'ILMN_1651230', 'ILMN_1651232', 'ILMN_1651236',
       'ILMN_1651238', 'ILMN_1651253', 'ILMN_1651254', 'ILMN_1651259',
       'ILMN_1651260', 'ILMN_1651262', 'ILMN_1651268', 'ILMN_1651278',
       'ILMN_1651281', 'ILMN_1651282', 'ILMN_1651285', 'ILMN_1651286'],
      dtype='object', name='ID')

Gene expression data available: True


### Step 4: Gene Identifier Review

In [5]:
# These identifiers are Illumina BeadChip IDs (ILMN_xxxxxxx), not human gene symbols
# They need to be mapped to standard gene symbols for downstream analysis
requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Extract gene annotation data from the SOFT file
print("Extracting gene annotation data from SOFT file...")
try:
    # Use the library function to extract gene annotation
    gene_annotation = get_gene_annotation(soft_file)
    print(f"Successfully extracted gene annotation data with {len(gene_annotation.index)} rows")
    
    # Preview the annotation DataFrame
    print("\nGene annotation preview (first few rows):")
    print(preview_df(gene_annotation))
    
    # Show column names to help identify which columns we need for mapping
    print("\nColumn names in gene annotation data:")
    print(gene_annotation.columns.tolist())
    
    # Check for relevant mapping columns
    if 'GB_ACC' in gene_annotation.columns:
        print("\nThe dataset contains GenBank accessions (GB_ACC) that could be used for gene mapping.")
        # Count non-null values in GB_ACC column
        non_null_count = gene_annotation['GB_ACC'].count()
        print(f"Number of rows with GenBank accessions: {non_null_count} out of {len(gene_annotation)}")
    
    if 'SPOT_ID' in gene_annotation.columns:
        print("\nThe dataset contains genomic regions (SPOT_ID) that could be used for location-based gene mapping.")
        print("Example SPOT_ID format:", gene_annotation['SPOT_ID'].iloc[0])
    
except Exception as e:
    print(f"Error processing gene annotation data: {e}")
    is_gene_available = False


Extracting gene annotation data from SOFT file...


Successfully extracted gene annotation data with 1498611 rows

Gene annotation preview (first few rows):
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Species': [nan, nan, nan, nan, nan], 'Source': [nan, nan, nan, nan, nan], 'Search_Key': [nan, nan, nan, nan, nan], 'Transcript': [nan, nan, nan, nan, nan], 'ILMN_Gene': [nan, nan, nan, nan, nan], 'Source_Reference_ID': [nan, nan, nan, nan, nan], 'RefSeq_ID': [nan, nan, nan, nan, nan], 'Unigene_ID': [nan, nan, nan, nan, nan], 'Entrez_Gene_ID': [nan, nan, nan, nan, nan], 'GI': [nan, nan, nan, nan, nan], 'Accession': [nan, nan, nan, nan, nan], 'Symbol': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB'], 'Protein_Product': [nan, nan, nan, nan, 'thrB'], 'Probe_Id': [nan, nan, nan, nan, nan], 'Array_Address_Id': [5090180.0, 6510136.0, 7560739.0, 1450438.0, 1240647.0], 'Probe_Type': [nan, nan, nan, nan, nan], 'Probe_Start': [nan, nan, nan, nan

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Identify which columns in gene_annotation contain probe IDs and gene symbols
print("Identifying relevant columns for gene mapping...")

# The gene expression data uses 'ILMN_xxxxxxx' IDs as shown in a previous step
# From the gene_annotation preview, we see:
# - 'ID' column contains the same ILMN IDs that match gene_data index
# - 'Symbol' column contains gene symbols we want to map to

# 2. Create gene mapping dataframe using the identified columns
prob_col = 'ID'  # Column containing probe IDs
gene_col = 'Symbol'  # Column containing gene symbols

print(f"Creating gene mapping from {prob_col} to {gene_col}...")
mapping_data = get_gene_mapping(gene_annotation, prob_col, gene_col)

print(f"Generated mapping data with {len(mapping_data)} entries")
print("First few rows of mapping data:")
print(preview_df(mapping_data))

# 3. Apply gene mapping to convert probe-level data to gene-level expression
print("Converting probe-level measurements to gene expression data...")
gene_data = apply_gene_mapping(gene_data, mapping_data)

print(f"Successfully converted to gene expression data with {len(gene_data)} genes")
print("First few gene symbols:")
print(gene_data.index[:10].tolist())

# Optional: normalize gene symbols to handle synonyms
print("Normalizing gene symbols...")
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"After normalization: {len(gene_data)} unique genes")
print("First few normalized gene symbols:")
print(gene_data.index[:10].tolist())

# Save the gene data to file
out_gene_dir = os.path.dirname(out_gene_data_file)
os.makedirs(out_gene_dir, exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


Identifying relevant columns for gene mapping...
Creating gene mapping from ID to Symbol...
Generated mapping data with 44837 entries
First few rows of mapping data:
{'ID': ['ILMN_1343048', 'ILMN_1343049', 'ILMN_1343050', 'ILMN_1343052', 'ILMN_1343059'], 'Gene': ['phage_lambda_genome', 'phage_lambda_genome', 'phage_lambda_genome:low', 'phage_lambda_genome:low', 'thrB']}
Converting probe-level measurements to gene expression data...
Successfully converted to gene expression data with 17999 genes
First few gene symbols:
['A1CF', 'A26C3', 'A2BP1', 'A2LD1', 'A2M', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS']
Normalizing gene symbols...


After normalization: 17405 unique genes
First few normalized gene symbols:
['A1BG-AS1', 'A1CF', 'A2M', 'A3GALT2', 'A4GALT', 'A4GNT', 'AAA1', 'AAAS', 'AACS', 'AADAC']


Gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE95425.csv


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols in the gene expression data
print("\nNormalizing gene symbols...")
try:
    normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
    print(f"After normalization: {len(normalized_gene_data.index)} unique gene symbols")
    
    # Save the normalized gene data (this was already done in Step 6, but we'll ensure it's saved)
    os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
    normalized_gene_data.to_csv(out_gene_data_file)
    print(f"Normalized gene expression data saved to {out_gene_data_file}")
except Exception as e:
    print(f"Error normalizing gene symbols: {e}")
    normalized_gene_data = gene_data  # Use original data if normalization fails

# Respect the determination made in Step 2 that trait data is not available
print("\nChecking trait availability from previous analysis...")
# We previously determined trait_row is None and is_trait_available is False
if trait_row is None:
    print("Clinical trait data not available, skipping clinical feature extraction.")
    is_trait_available = False
    clinical_df = None
else:
    # This block would execute if trait data was available, which it isn't in this case
    print("Extracting clinical features...")
    try:
        clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        print(f"Clinical data extracted with shape: {clinical_df.shape}")
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        clinical_df.to_csv(out_clinical_data_file)
        print(f"Clinical data saved to {out_clinical_data_file}")
        is_trait_available = True
    except Exception as e:
        print(f"Error extracting clinical features: {e}")
        is_trait_available = False
        clinical_df = None

# 2. Link clinical and genetic data if available
print("\nLinking clinical and genetic data...")
try:
    if clinical_df is not None and not normalized_gene_data.empty:
        # Print sample IDs from both datasets for debugging
        print("First few clinical sample columns:", list(clinical_df.columns)[:5])
        print("First few genetic sample columns:", list(normalized_gene_data.columns)[:5])
        
        # Link clinical and genetic data
        linked_data = geo_link_clinical_genetic_data(clinical_df, normalized_gene_data)
        print(f"Linked data shape: {linked_data.shape}")
        
        # Check if we have at least one sample with trait value
        trait_count = linked_data[trait].count()
        print(f"Number of samples with trait values: {trait_count}")
        
        if trait_count > 0:
            # 3. Handle missing values systematically
            print("\nHandling missing values...")
            linked_data = handle_missing_values(linked_data, trait)
            print(f"After handling missing values, data shape: {linked_data.shape}")
            
            # Check if we still have samples after missing value handling
            if linked_data.shape[0] > 0:
                # 4. Determine whether the trait and demographic features are biased
                print("\nChecking for bias in features...")
                is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
            else:
                print("Error: All samples were removed during missing value handling.")
                is_biased = True
        else:
            print("No samples have valid trait values. Dataset cannot be used.")
            is_biased = True
    else:
        print("Cannot link data: clinical or genetic data is missing")
        linked_data = pd.DataFrame()
        is_biased = True
        
except Exception as e:
    print(f"Error in linking clinical and genetic data: {e}")
    linked_data = pd.DataFrame()
    is_biased = True

# 5. Final quality validation
print("\nPerforming final validation...")
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased if 'is_biased' in locals() else True,
    df=linked_data if 'linked_data' in locals() and not linked_data.empty else pd.DataFrame(),
    note="Dataset contains gene expression data from kidney tissue samples but lacks trait information needed for association studies."
)

# 6. Save linked data if usable
if is_usable and 'linked_data' in locals() and not linked_data.empty:
    # Create directory if it doesn't exist
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    
    # Save linked data
    linked_data.to_csv(out_data_file)
    print(f"Linked data saved to {out_data_file}")
else:
    print(f"Dataset not usable for {trait} association studies. Data not saved.")


Normalizing gene symbols...
After normalization: 17405 unique gene symbols


Normalized gene expression data saved to ../../output/preprocess/Kidney_Clear_Cell_Carcinoma/gene_data/GSE95425.csv

Checking trait availability from previous analysis...
Clinical trait data not available, skipping clinical feature extraction.

Linking clinical and genetic data...
Cannot link data: clinical or genetic data is missing

Performing final validation...
Abnormality detected in the cohort: GSE95425. Preprocessing failed.
Dataset not usable for Kidney_Clear_Cell_Carcinoma association studies. Data not saved.
